<h1>Conceito<h1>

<h2>K-means</h2>

O algoritmo K-means é um agrupador não supervisionado que agrupa os dados ao tentar separar as amostras em n grupos de igual variância, minimizando um critério conhecido como inércia ou soma dos quadrados dentro dos clusters.

O algoritmo divide uma amostra de $N$ elementos $X$ em $K$ grupos disjuntos $C$, cada um deles descritos pela média $\mu_j$ das amostras no grupo. As médias são normalmentes chamadas de **centróides** dos grupos. Note que eles não são necessariamentes pontos reais $X$.

O K-means busca **escolher centróides $\mu_j$ que minimizem a inércia**:

$\large \sum\limits_{i=0}^{n} \ \min\limits_{\mu_j \in C} (||x_i - \mu_j||²)$

A inércia pode ser descrita como uma medidade de quão internamente coerente os agrupamentos são.

A inércia porém tem algumas **limitações**:
- Ela faz a suposição que os agrupamentos são convexos e isotrópicos, o que não é sempre o caso. Portanto ela lida mal com agrupamentos alongados ou com formas irregulares
- A inércia não é uma métrica normalizada, só é possível levar em consideração que, quanto mais próximo de zero, melhor. Mas em ambientes de alta dimensionalidade, a inércia pode se tornar 'inflacionada'. Nesses casos, pode ser interessante realizar uma **redução de dimensionalidade**.

<h1>Aplicação</h1>

In [21]:
# Importando as bibliotecas que serão utilizadas
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import plotly.express as px

In [2]:
# Importando a base de dados
base_genero = pd.read_csv(r'Dados/data_by_genres.csv')
base_genero = base_genero.drop(['mode','key'], axis = 1)

In [11]:
# Definindo random seed
SEED = 20

np.random.seed(SEED)

# Criando uma pipeline para padronizar os dados e reduzir a dimensionalidade
pca_pipeline = Pipeline(
    [
    ('scaler', StandardScaler()),
    ('PCA', PCA(n_components=2))
    ]
)

genre_embedding_pca = pca_pipeline.fit_transform(base_genero[base_genero.columns[1:]])

projection = pd.DataFrame(columns=['x', 'y'], data = genre_embedding_pca)

In [17]:
# Setando a random seed novamente via numpy
np.random.seed(SEED)

# Criando o modelo kmeans
kmeans_pca = KMeans(n_clusters=5, verbose=True)

kmeans_pca.fit(projection)

# Salvando os resultados nos dataframes
base_genero['cluster_pca'] = kmeans_pca.predict(projection)
projection['cluster_pca'] = base_genero['cluster_pca'].copy()

c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Initialization complete
Iteration 0, inertia 4821.073472299924.
Iteration 1, inertia 3820.9549122471653.
Iteration 2, inertia 3686.626780219592.
Iteration 3, inertia 3619.314769699243.
Iteration 4, inertia 3575.4040262662024.
Iteration 5, inertia 3554.047907992452.
Iteration 6, inertia 3545.1571295405925.
Iteration 7, inertia 3537.1496480477213.
Iteration 8, inertia 3526.0721086571893.
Iteration 9, inertia 3519.4913803930103.
Iteration 10, inertia 3516.5434371361985.
Iteration 11, inertia 3513.714764396059.
Iteration 12, inertia 3510.2562635552854.
Iteration 13, inertia 3505.9117875816873.
Iteration 14, inertia 3502.663987650568.
Iteration 15, inertia 3501.4040648217015.
Iteration 16, inertia 3500.8205717654905.
Iteration 17, inertia 3500.564594414561.
Converged at iteration 17: center shift 9.720147618834785e-05 within tolerance 0.00027098868470996516.
Initialization complete
Iteration 0, inertia 5359.623966918487.
Iteration 1, inertia 3859.7624189001144.
Iteration 2, inertia 3613.660

In [20]:
projection['genres'] = base_genero['genres'].copy()

In [22]:
fig = px.scatter(
    projection, x='x', y='y', color='cluster_pca', hover_data=['x', 'y', 'genres']
)
fig.show()

In [23]:
# Verificando o quanto o pca explica a variância é explicada pelo PCA
pca_pipeline[1].explained_variance_ratio_

array([0.34986105, 0.14284565])

In [24]:
# Verificando o quanto o pca explica a variância é explicada pelo PCA
pca_pipeline[1].explained_variance_

array([3.84976644, 1.57183087])

Ou seja, das 11 colunas iniciais, o modelo explica aproximadamente 5 delas.